In [20]:
import os
import yaml
import argparse
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv

from download import download_ticker
from database import (
    get_connection, 
    upsert_raw_series, 
    log_ingestion_run,
    close_connection
)

In [21]:
def load_universe():
    """Load tickers from universe.yml"""
    universe_path = Path.cwd().parent.parent / 'configs' / 'universe.yml'
    
    with open(universe_path, 'r') as f:
        universe = yaml.safe_load(f)
    
    # Flatten into list
    tickers = []
    for asset_class, ticker_list in universe.items():
        for item in ticker_list:
            tickers.append({
                'ticker': item['ticker'],
                'name': item['name'],
                'source': item['source'],
                'asset_class': asset_class.upper()
            })
    
    return tickers

In [22]:
def ingest_all(start_date='2010-01-01', ticker_filter=None, dry_run=False):
    """
    Main ingestion function.
    
    Args:
        start_date: Start date for historical data
        ticker_filter: If provided, only ingest this ticker
        dry_run: If True, download but don't store in database
    """
    # Load environment
    load_dotenv()
    fred_api_key = os.getenv('FRED_API_KEY')
    db_password = os.getenv('DB_PASSWORD')
    
    # Load universe
    tickers = load_universe()
    
    if ticker_filter:
        tickers = [t for t in tickers if t['ticker'] == ticker_filter]
    
    print(f"Ingesting {len(tickers)} tickers...")
    
    # Connect to database
    conn = None if dry_run else get_connection(password=db_password)
    
    # Start ingestion log
    run_id = datetime.now().strftime('%Y%m%d_%H%M%S')
    if not dry_run:
        log_ingestion_run(conn, run_id, 'running', f'Ingesting {len(tickers)} tickers')
    
    success_count = 0
    fail_count = 0
    
    # Process each ticker
    for ticker_info in tickers:
        ticker = ticker_info['ticker']
        source = ticker_info['source']
        asset_class = ticker_info['asset_class']
        
        print(f"Processing {ticker} ({asset_class}) from {source}...")
        
        # Download
        df = download_ticker(ticker, source, fred_api_key, start_date)
        
        if df is not None and not df.empty:
            if not dry_run:
                # Store in database
                row_count = upsert_raw_series(conn, df, asset_class)
                print(f"  ✓ Inserted {row_count} rows")
            else:
                print(f"  ✓ Downloaded {len(df)} rows (dry run - not stored)")
            
            success_count += 1
        else:
            print(f"  ✗ Failed to download")
            fail_count += 1
    
    # Finish ingestion log
    if not dry_run:
        log_ingestion_run(
            conn, run_id, 'ok', 
            f'Success: {success_count}, Failed: {fail_count}'
        )
        close_connection(conn)
    
    print(f"\nIngestion complete! Success: {success_count}, Failed: {fail_count}")

In [23]:
ticker_filter = '^GSPC'
start_date='2010-01-01'

ingest_all(
        start_date=start_date,
        ticker_filter=ticker_filter,
        dry_run=True
    )

Ingesting 1 tickers...
Processing ^GSPC (EQUITY) from yfinance...
Error downloading ^GSPC from yfinance: Data must be 1-dimensional, got ndarray of shape (4012, 1) instead
  ✗ Failed to download

Ingestion complete! Success: 0, Failed: 1


/Users/alecmitchell-thomson/Desktop/Coding/macro-regime-engine/src/ingestion/download.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data: pd.DataFrame = pd.DataFrame(yf.download(ticker, start=start_date, progress=False))
